Transfer learning with Xception model.
Cross Validation to create 4 models and used average prediction for submission.
Slightly tweaked the normalisation means of the images.

In [1]:
import numpy as np
import pandas as pd
from glob import glob
from os.path import join
from pathlib import Path
from PIL import Image
import matplotlib.pyplot as plt
from tqdm import tqdm
from torchvision.transforms import ToPILImage, Grayscale, Compose, Resize, ToTensor, Normalize, RandomHorizontalFlip, RandomRotation
import torch
import torch.nn as nn
import torchvision
from torchvision.transforms import Compose, Resize, CenterCrop, ToTensor, Normalize
import torch.optim as optim
from itertools import permutations
from torchvision.models import EfficientNet
import timm
from sklearn.model_selection import KFold
from torchvision.models import vision_transformer

In [2]:
def get_device():
  """
  Gets the device to use for training (GPU if available)
  """
  return torch.device("cuda" if torch.cuda.is_available() else "cpu")

device = get_device()

In [3]:
class AgeDataset(torch.utils.data.Dataset):

    def __init__(self, data_path, annot_path, train=True):
        super(AgeDataset, self).__init__()

        self.annot_path = annot_path
        self.data_path = data_path
        self.train = train

        self.ann = pd.read_csv(annot_path)
        self.files = self.ann['file_id']
        if train:
            self.ages = self.ann['age']
        self.transform = self._transform(224)

    @staticmethod    
    def _convert_image_to_rgb(image):
        return image.convert("RGB")

    def _transform(self, n_px):
        mean = [0.7, 0.5, 0.3]
        std = [0.229, 0.224, 0.225]
        return Compose([
            Resize(n_px),
            self._convert_image_to_rgb,
            ToTensor(),
            Normalize(mean, std),
        ])

    def read_img(self, file_name):
        im_path = join(self.data_path,file_name)   
        img = Image.open(im_path)
        img = self.transform(img)
        return img

    def __getitem__(self, index):
        file_name = self.files[index]
        img = self.read_img(file_name)
        if self.train:
            age = self.ages[index]
            return img, age
        else:
            return img

    def __len__(self):
        return len(self.files)

In [4]:
train_path = '/kaggle/input/smai-24-age-prediction/content/faces_dataset/train'
train_ann = '/kaggle/input/smai-24-age-prediction/content/faces_dataset/train.csv'
train_dataset = AgeDataset(train_path, train_ann, train=True)



test_path = '/kaggle/input/smai-24-age-prediction/content/faces_dataset/test'
test_ann = '/kaggle/input/smai-24-age-prediction/content/faces_dataset/submission.csv'
test_dataset = AgeDataset(test_path, test_ann, train=False)

In [5]:
# train_size = int(0.9 * len(train_dataset))
# valid_size = len(train_dataset) - train_size
# train_dataset, valid_dataset = torch.utils.data.random_split(train_dataset, [train_size, valid_size])


In [6]:
# train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)
# valid_loader = torch.utils.data.DataLoader(valid_dataset, batch_size=64, shuffle=True)
# test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=64, shuffle=False)

In [7]:

# # Define the Feature Extractor with transfer learning for age regression
# class FeatureExtractor(nn.Module):
#     def __init__(self):
#         super(FeatureExtractor, self).__init__()
#         self.inceptionv4 = timm.create_model('xception', pretrained=True)
#         print(self.inceptionv4)
#         self.inceptionv4.fc = nn.Linear(2048, 1)

#     def forward(self, x):
#         x = self.inceptionv4(x)  
#         return x

# # Initialize the feature extractor with transfer learning
# feature_extractor = FeatureExtractor()
# feature_extractor.to(device)

# # Define loss function and optimizer for regression (e.g., Mean Squared Error loss)
# criterion = nn.MSELoss()
# optimizer = optim.Adam(feature_extractor.parameters(), lr=0.001)

# best_mse=10000000
# beat_model=None
# epochs=15

# # Training loop
# num_epochs = 15 
# for epoch in range(num_epochs):
#     feature_extractor.train()
#     running_loss = 0.0
#     mae=0
#     mse=0
#     num_examples=0
#     pbar = tqdm(train_loader, desc=f'Epoch [{epoch+1}/{num_epochs}]', leave=False)
#     for images, ages in pbar:
#         images, ages = images.to(device), ages.to(device)
#         optimizer.zero_grad()
#         outputs = feature_extractor(images)
#         loss = criterion(outputs.squeeze(), ages.float())  # Compute loss for age regression
#         loss.backward()
#         optimizer.step()
#         running_loss += loss.item() * images.size(0)
#         num_examples += ages.size(0)
#         mae += torch.sum(torch.abs(outputs.squeeze() - ages.float()))
#         mse += torch.sum((outputs.squeeze() - ages.float())**2)
#         pbar.set_postfix({'Loss': loss.item()})
        
#     mae = mae.float() / num_examples
#     mse = mse.float() / num_examples
#     print(epoch+1,"Train data",mae,mse)
    
#     epoch_loss = running_loss / len(train_loader.dataset)
# #     print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}')
    
#     feature_extractor.eval()
    
#     mae=0
#     mse=0
#     num_examples=0
    
#     with torch.no_grad():
#         pbar = tqdm(valid_loader, desc=f'Epoch [{epoch+1}/{num_epochs}]', leave=False)
#         for images, ages in pbar:
#             images, ages = images.to(device), ages.to(device)
#             outputs = feature_extractor(images)
#             loss = criterion(outputs.squeeze(), ages.float()) 
#             running_loss += loss.item() * images.size(0)
#             num_examples += ages.size(0)
#             mae += torch.sum(torch.abs(outputs.squeeze() - ages.float()))
#             mse += torch.sum((outputs.squeeze() - ages.float())**2)
#             pbar.set_postfix({'Loss': loss.item()})

#     mae = mae.float() / num_examples
#     mse = mse.float() / num_examples
#     print(epoch+1,"Valid data",mae,mse)
    
#     if(mse<best_mse):
#         best_mse=mse
#         best_model=feature_extractor
        
        
        
# print(best_mse)
        

In [8]:
# feature_extractor=best_model
# # Initialize an empty list to store the predicted ages
# predicted_ages = []

# # Evaluate the model on the test set
# feature_extractor.eval()
# with torch.no_grad():
#     for images in test_loader:
#         images = images.to(device)
#         outputs = feature_extractor(images)
#         predicted_ages.extend(outputs.squeeze().cpu().numpy())

# # Now 'predicted_ages' contains the predicted ages for the images in the test set
# # print(y_pred)
# submit = pd.read_csv('/kaggle/input/smai-24-age-prediction/content/faces_dataset/submission.csv')
# submit['age'] = predicted_ages
# submit.head()

# print(submit['age'])

# submit.to_csv('baseline.csv',index=False)

In [9]:
# Define the number of folds
n_splits = 4

# Initialize KFold
kf = KFold(n_splits=n_splits, shuffle=True)

# Initialize best_model list to store the best model for each fold
best_model = [None] * n_splits

# Define loss function and optimizer for regression (e.g., Mean Squared Error loss)
criterion = nn.MSELoss()


test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=64, shuffle=False)

In [10]:
# Define the Feature Extractor with transfer learning for age regression
class FeatureExtractor(nn.Module):
    def __init__(self):
        super(FeatureExtractor, self).__init__()
        self.inceptionv4 = timm.create_model('xception', pretrained=True)
#         print(self.inceptionv4)
        self.inceptionv4.fc = nn.Linear(2048, 1)

    def forward(self, x):
        x = self.inceptionv4(x)  
        return x

In [11]:
# Training loop with N-fold cross-validation
for fold, (train_index, valid_index) in enumerate(kf.split(train_dataset)):
    print(f"Fold [{fold+1}/{n_splits}]")
    
    
    # Create the data loaders for this fold
    train_subset = torch.utils.data.Subset(train_dataset, train_index)
    valid_subset = torch.utils.data.Subset(train_dataset, valid_index)
    train_loader = torch.utils.data.DataLoader(train_subset, batch_size=64, shuffle=True)
    valid_loader = torch.utils.data.DataLoader(valid_subset, batch_size=64, shuffle=False)
    
    # Initialize the model
    feature_extractor = FeatureExtractor()
    feature_extractor.to(device)
    optimizer = optim.Adam(feature_extractor.parameters())
    
    best_mse = 10000000
    
    num_epochs=15
    
    # Training loop for this fold
    for epoch in range(num_epochs):
        # Training phase
        feature_extractor.train()
        running_loss = 0.0
        pbar_train = tqdm(train_loader, desc=f'Epoch [{epoch+1}/{num_epochs}], Fold [{fold+1}/{n_splits}] - Training', leave=False)
        for images, ages in pbar_train:
            images, ages = images.to(device), ages.to(device)
            optimizer.zero_grad()
            outputs = feature_extractor(images)
            loss = criterion(outputs.squeeze(), ages.float())
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * images.size(0)
            pbar_train.set_postfix({'Train Loss': loss.item()})
        epoch_loss = running_loss / len(train_loader.dataset)
        
        
        # Validation phase
        feature_extractor.eval()
        valid_mse = 0
        valid_examples = 0
        pbar_valid = tqdm(valid_loader, desc=f'Epoch [{epoch+1}/{num_epochs}], Fold [{fold+1}/{n_splits}] - Validation', leave=False)
        with torch.no_grad():
            for images, ages in pbar_valid:
                images, ages = images.to(device), ages.to(device)
                outputs = feature_extractor(images)
                valid_mse += torch.sum((outputs.squeeze() - ages.float()) ** 2).item()
                valid_examples += ages.size(0)
                pbar_valid.set_postfix({'Valid Loss': valid_mse / valid_examples})
        valid_mse /= valid_examples
        
        print(f"Epoch [{epoch+1}/{num_epochs}], Fold [{fold+1}/{n_splits}], Train Loss: {epoch_loss:.4f}, Valid Loss: {valid_mse:.4f}")
        
        # Update best model
        if valid_mse < best_mse:
            best_mse = valid_mse
            best_model[fold] = feature_extractor
            
    print(f"Best MSE for this fold: {best_mse:.4f}")


Fold [1/4]


/opt/conda/lib/python3.10/site-packages/timm/models/_factory.py:117: UserWarning: Mapping deprecated model name xception to current legacy_xception.
  model = create_fn(
Downloading: "https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-cadene/xception-43020ad28.pth" to /root/.cache/torch/hub/checkpoints/xception-43020ad28.pth


Epoch [1/15], Fold [1/4], Train Loss: 201.0045, Valid Loss: 62.3279


Epoch [2/15], Fold [1/4], Train Loss: 47.5944, Valid Loss: 54.5193


Epoch [3/15], Fold [1/4], Train Loss: 32.6064, Valid Loss: 57.0444


...

Epoch [11/15], Fold [4/4], Train Loss: 13.3088, Valid Loss: 50.9366


Epoch [12/15], Fold [4/4], Train Loss: 8.2394, Valid Loss: 69.0594


Epoch [13/15], Fold [4/4], Train Loss: 5.9318, Valid Loss: 56.2326


Epoch [14/15], Fold [4/4], Train Loss: 12.2936, Valid Loss: 48.8501


Epoch [15/15], Fold [4/4], Train Loss: 7.9856, Valid Loss: 51.9499
Best MSE for this fold: 48.4183


In [14]:
# Assuming n_splits (number of folds) is 4 and you want the code for the 4th fold (fold_num = 3)

fold_num = 3  # Change this to the desired fold number (0-indexed)

# Assuming you have already performed the split using `kf.split(train_dataset)` and stored the indices
train_index, valid_index = list(kf.split(train_dataset))[fold_num]

# Create the data loaders for the 4th fold
train_subset = torch.utils.data.Subset(train_dataset, train_index)
valid_subset = torch.utils.data.Subset(train_dataset, valid_index)
train_loader = torch.utils.data.DataLoader(train_subset, batch_size=64, shuffle=True)
valid_loader = torch.utils.data.DataLoader(valid_subset, batch_size=64, shuffle=False)

# Initialize the model (assuming you have a function to define it)
feature_extractor = FeatureExtractor()
feature_extractor.to(device)
optimizer = optim.Adam(feature_extractor.parameters())

best_mse = 10000000

num_epochs = 15

# Training loop for the 4th fold
for epoch in range(num_epochs):
    # Training phase
    feature_extractor.train()
    running_loss = 0.0
    pbar_train = tqdm(train_loader, desc=f'Epoch [{epoch+1}/{num_epochs}], Fold [{fold_num+1}/{n_splits}] - Training', leave=False)
    for images, ages in pbar_train:
        images, ages = images.to(device), ages.to(device)
        optimizer.zero_grad()
        outputs = feature_extractor(images)
        loss = criterion(outputs.squeeze(), ages.float())
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * images.size(0)
        pbar_train.set_postfix({'Train Loss': loss.item()})
    epoch_loss = running_loss / len(train_loader.dataset)

    # Validation phase
    feature_extractor.eval()
    valid_mse = 0
    valid_examples = 0
    pbar_valid = tqdm(valid_loader, desc=f'Epoch [{epoch+1}/{num_epochs}], Fold [{fold_num+1}/{n_splits}] - Validation', leave=False)
    with torch.no_grad():
        for images, ages in pbar_valid:
            images, ages = images.to(device), ages.to(device)
            outputs = feature_extractor(images)
            valid_mse += torch.sum((outputs.squeeze() - ages.float()) ** 2).item()
            valid_examples += ages.size(0)
            pbar_valid.set_postfix({'Valid Loss': valid_mse / valid_examples})
    valid_mse /= valid_examples

    print(f"Epoch [{epoch+1}/{num_epochs}], Fold [{fold_num+1}/{n_splits}], Train Loss: {epoch_loss:.4f}, Valid Loss: {valid_mse:.4f}")

    # You don't need to update the best model here since you're training a single fold
    # This functionality would be relevant if you were training all folds in a loop
    # Update best model
    if valid_mse < best_mse:
        best_mse = valid_mse
        best_model[fold_num] = feature_extractor

print(f"**Training completed for Fold {fold_num+1}**")

/opt/conda/lib/python3.10/site-packages/timm/models/_factory.py:117: UserWarning: Mapping deprecated model name xception to current legacy_xception.
  model = create_fn(


Epoch [1/15], Fold [4/4], Train Loss: 196.8904, Valid Loss: 59.9815


Epoch [2/15], Fold [4/4], Train Loss: 47.9628, Valid Loss: 61.6433


Epoch [3/15], Fold [4/4], Train Loss: 33.9115, Valid Loss: 73.8384


Epoch [4/15], Fold [4/4], Train Loss: 24.1256, Valid Loss: 53.2982


...

Epoch [11/15], Fold [4/4], Train Loss: 8.9518, Valid Loss: 49.2627


Epoch [12/15], Fold [4/4], Train Loss: 8.9271, Valid Loss: 62.3070


Epoch [13/15], Fold [4/4], Train Loss: 8.2557, Valid Loss: 45.9108


Epoch [14/15], Fold [4/4], Train Loss: 7.3656, Valid Loss: 52.1545


Epoch [15/15], Fold [4/4], Train Loss: 7.0798, Valid Loss: 58.1772
**Training completed for Fold 4**


In [15]:
# Initialize an empty list to store predictions from all models
test_predictions = []

# Testing loop with the best models from each fold
for fold, (_, valid_index) in enumerate(kf.split(train_dataset)):
    # Extract the best model for this fold
    feature_extractor = best_model[fold]
    feature_extractor.eval()
    
    fold_predictions = []  # Store predictions for this fold
    
    with torch.no_grad():
        for images in tqdm(test_loader):
            images = images.to(device)
            outputs = feature_extractor(images)
            fold_predictions.extend(outputs.squeeze().cpu().numpy())
    
    test_predictions.append(fold_predictions)

# Convert the list of predictions into a NumPy array for easier manipulation
test_predictions = np.array(test_predictions)

# Calculate the average prediction across all models
average_predictions = np.mean(test_predictions, axis=0)
# print(len(average_predictions))
submit = pd.read_csv('/kaggle/input/smai-24-age-prediction/content/faces_dataset/submission.csv')
submit['age'] = average_predictions
submit.head()

print(submit['age'])

submit.to_csv('baseline.csv',index=False)


100%|██████████| 31/31 [00:11<00:00,  2.79it/s]

0       61.728020
1       46.289104
2       32.876629
3       59.208015
4       21.013227
          ...    
1945     1.038210
1946    37.729767
1947    32.808361
1948     3.902917
1949    53.834087
Name: age, Length: 1950, dtype: float32
